In [33]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json
from pandas.io.json import json_normalize

pd.options.display.max_colwidth = 100

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?p ?o
    WHERE { <http://dbpedia.org/resource/Martin_Scorsese> ?p ?o }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results_df = json_normalize(results["results"]["bindings"])
results_df = results_df[results_df['o.type'] == "uri"]
results_df = results_df.drop(['o.datatype','o.type', 'o.xml:lang','p.type'],axis=1)
results_df = results_df[['p.value','o.value']]
results_df
# results
# for result in results["results"]["bindings"]:
#     print(result["p"]["value"] + " : " + result["o"]["value"]) 
    
# review_df = pd.DataFrame(sparql,index=[0])
# review_df.head()


,p.value,o.value
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#Thing
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://xmlns.com/foaf/0.1/Person
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/ontology/Person
3,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#Agent
4,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson
5,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.wikidata.org/entity/Q215627
6,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.wikidata.org/entity/Q24229398
7,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.wikidata.org/entity/Q5
8,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/ontology/Agent
9,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Person
